# MSII Solution Process

## About:
The Design Project Cadet Handout allows for "a spreadsheet, computer program (e.g., MatLab) or hand calculations to support your analysis," provided that your final submission is "formatted in such a way that the instructor can see the formulas you are using and the final answers." Python is a computational tool of a very similar caliber to MatLab (in the hands of an experienced user), and is thus authorized. A brief in-class discussion with Gen. Thompson confirmed this.

I have, to this end, written a fairly comprehensive software library which is capable of computing, among other things, the percent coverage, budget calculations, orbit positions by time, viability assessments, and many other rudimentary computations of a given constellation. The exact code used for the entire project will be included in the submission, as well as a condensed version with the exact formulas we used. 

Understand that the code in this notebook will be extremely abstracted and surface level because it relies on the software library (entirely written by me) for all the computations that would truly be considered Astro-related.



## MS II Checklist:
### Pre-Checklist Constellation Computations:

#### Import all the necessary code from our custom software library:

In [1]:
# handle all the necessary imports from our custom software library
from MSII_Components import *
from MSII_useful_consts import *
from MSII_Constellation import MSII_Constellation
from MSII_Orbit import MSII_Orbit
from MSII_Constraints import *
from Sensing_Formulas import *

#### Create a modifyable template of our MSI constellation (already has good coverage):

In [2]:
def create_MSI_constellation(payload, adcs, structure):
    msI_a = GEOSYNCHRONOUS_SEMIMAJOR_AXIS # 42241km from MSI
    max_height = get_payload_max_height(payload, MIN_RESOLUTION) # how high can we get and still hit our
    # target resolution (formula derivation done and included on paper) in km

    # from formula sheet: Ra = a(1 + e)
    # it follows: e = (Ra / a) - 1 = ((h + Rearth) / a) - 1
    max_eccentricity = ((max_height + R_EARTH) / msI_a) - 1
    max_eccentricity = abs(max_eccentricity)

    return { 
        "payload": payload,
        "adcs": adcs,
        "structure": structure,
        "sats": [ # we update all the fields that will change with new MSII requirements (just e)
            {
                "a": msI_a,
                "e": max_eccentricity,
                "i": 50.825, # from MSI
                "raan": 340.5, # from MSI
                "w": 245, # from MSI
                "v": 180 # from MSI
            }, {
                "a": msI_a,
                "e": max_eccentricity,
                "i": 50.825, # from MSI
                "raan": 100.5, # from MSI
                "w": 245, # from MSI
                "v": 154.5099795 # from MSI
            }, {
                "a": msI_a,
                "e": max_eccentricity,
                "i": 50.825, # from MSI
                "raan": 220.5, # from MSI
                "w": 245, # from MSI
                "v": 205.4900205 # from MSI
            }
        ]
    }

#### Generate all possible hardware configurations

In [3]:
constellation_options = []
for payload in PAYLOADS: # find every possible payload-adcs-structure combination
    for adcs in ADCS:
        for struct in STRUCTURES:
            # apply our template from before (which includes modified e computations)
            constellation_object = MSII_Constellation(create_MSI_constellation(payload, adcs, struct))
            constellation_options.append(constellation_object)

# output how many we found
print(f"Found {len(constellation_options)} possible constellations.")

viable_constellation_options = []
for constellation in constellation_options:
    # filter out all the ones that obviously fail (i.e. out of budget, crash into earth, fail volume constraints)
    if (constellation.get_concise_summary().passes_inspection):
        viable_constellation_options.append(constellation)

# output how many we found
print(f"Found {len(viable_constellation_options)} viable constellation options")

Found 36 possible constellations.
Found 15 viable constellation options


#### Figure out which one has the best coverage
Note that the algorithm for this is complicated and is included in the algorithm explanation portion of the project.

In [4]:
# we are having slight issues with our coverage computation algorithm, probably something to do with the
# fact that the Earth position and the orbits are out of sync 

best_coverage = 0 # assume zero coverage to start, update later
best_constellation = None

for constellation in viable_constellation_options:
    constellation : MSII_Constellation
    coverage_report = constellation.assess_coverage()
    if (not coverage_report.adcs_cam_compatible):
        # our pointing accuracy is so bad we cannot reliably point at anything
        # therefore, do not count this option's coverage in the contest
        continue

    if (coverage_report.percent_coverage > best_coverage):
        best_coverage = coverage_report.percent_coverage
        best_constellation = constellation


print(f"Found constellation with {best_coverage}% coverage")


# ===============================================================================================
# Our alternative is to find the one with the lowest eccentricity. This one will likely zoom slightly
# higher over the points and have high coverage because it spends more time on the side of the Earth
# that has the points.

best_e = 1
best_constellation = None

for constellation in viable_constellation_options:
    coverage_report = constellation.assess_coverage()
    if (not coverage_report.adcs_cam_compatible):
        # our pointing accuracy is so bad we cannot reliably point at anything
        # therefore, do not count this option's coverage in the contest
        continue

    eccentricity = constellation.sats[0].orbit.e
    if (eccentricity < best_e):
        best_e = eccentricity
        best_constellation = constellation

print(f"Found constellation with {best_e} eccentricity")
print(best_constellation.to_string())
print(best_constellation.get_comprehensive_summary().to_string("#1"))




Found constellation with 0% coverage
Found constellation with 0.8235576903856205 eccentricity
Constellation :
	Payload:   Astra Hi-Res Imager
	Structure: Option III
	ADCS:      Option II
	ORBIT: 
		a    : 42241.09773
		e    : 0.8235576903856205
		i    : 50.825
		raan : 340.5
		w    : 245
		v    : 180
	ORBIT: 
		a    : 42241.09773
		e    : 0.8235576903856205
		i    : 50.825
		raan : 100.5
		w    : 245
		v    : 154.5099795
	ORBIT: 
		a    : 42241.09773
		e    : 0.8235576903856205
		i    : 50.825
		raan : 220.5
		w    : 245
		v    : 205.4900205

Constellation #1 Comprehensive Summary: 
	Constellation #1 Concise Summary:
		Passes:       True
		In Budget:    True
		Viable Orbit: True
		Volume Fits:  True
	
	Constellation #1 Orbit Viability: VIABLE
		ORBIT 1
			CATASTROPHIC?      False
			DRAGLESS?          True
			CRASHLESS?         True
			Max Altitude:      70650.94161587209 km
			Min Altitude:      1074.9798441279218 km
			Is Geosynchronous: True
		
		ORBIT 2
			CATASTROPHIC?      False
